In [1]:
!pip install auto-sklearn
!pip install sweetviz
!pip install scipy

!pip list | grep scipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
scipy                         1.7.3


**Leitura dos dados e visualização de suas tabelas para eu eu possa saber qual será a variável alvo que no caso como quero prever a qualidade do vinho, será a 'quality'.**

In [2]:
import pandas as pd
df = pd.read_csv('/content/winequality-red.csv')
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


**Separação das features e da target.**

In [4]:
X = df.drop(columns='quality')
y = df.quality

**Separação do conjunto treino e teste, conjunto teste será de 30%, o treino 70%, nada fora do habitual.**

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size = 0.33)

**Nesta etapa importo o módulo de regressão da biblioteca de automl ''autosklearn'', e coloco alguns parâmetros básicos, nada muito complexo..**

In [6]:
import autosklearn.classification

model =  autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=120,per_run_time_limit=59, 
resampling_strategy='cv',
resampling_strategy_arguments={'folds': 10})

model.fit(X_train, y_train,dataset_name='df')


AutoSklearnClassifier(per_run_time_limit=59, resampling_strategy='cv',
                      resampling_strategy_arguments={'folds': 10},
                      time_left_for_this_task=120)

**Entre os diversos modelos treinados o que melhor performou foi o Random Forest, o interessante é que antes de utilizar a cross validation o melhor modelo foi o modelo Extra Trees, similar ao modelo utilizado, mas,  que utiliza mais da aleatoriedade.**

**Pode haver uma relação da implementação do cross-validation com a mudança de modelos tão similares, será algo que investigarei futuramente.**

In [7]:
print(model.leaderboard())
print(model.sprint_statistics())

          rank  ensemble_weight           type      cost   duration
model_id                                                           
4            1             0.64  random_forest  0.309991  20.164273
2            2             0.32  random_forest  0.320261  34.296169
3            3             0.04  random_forest  0.320261  45.793755
auto-sklearn results:
  Dataset name: df
  Metric: accuracy
  Best validation score: 0.690009
  Number of target algorithm runs: 4
  Number of successful target algorithm runs: 3
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0



**Aqui utilizei as métricas de R2 e a média de erro ao quadrado para que eu possa compreender melhor o meu modelo.**

In [8]:
import sklearn.metrics

X_train_predictions = model.predict(X_train)
X_test_predictions = model.predict(X_test)


print("Train R2 score:", sklearn.metrics.r2_score(y_train, X_train_predictions))
print("Test R2 score:", sklearn.metrics.r2_score(y_test, X_test_predictions))

print('Mean squared error:',sklearn.metrics.mean_squared_error(y_test,X_test_predictions))

Train R2 score: 1.0
Test R2 score: 0.312860160028234
Mean squared error: 0.44696969696969696
